In [21]:
# Importations nécessaires pour travailler avec Azure Machine Learning et Azure Storage Blob :
# 1. Importer les classes ComputeTarget et AmlCompute pour gérer les ressources de calcul dans Azure ML.
# 2. Importer les classes Workspace, Experiment, Dataset et Environment pour gérer l'espace de travail, les expériences, les jeux de données et les environnements dans Azure ML.
# 3. Importer les classes BlobServiceClient, BlobClient et ContainerClient pour interagir avec Azure Blob Storage, permettant la manipulation de conteneurs et de blobs.
# 4. Importer les classes ComputeTargetException pour gérer les exceptions spécifiques liées aux cibles de calcul dans Azure ML.
# 5. Importer la classe CondaDependencies pour spécifier les dépendances conda requises pour l'exécution des environnements.
# 6. Utiliser DEFAULT_CPU_IMAGE pour définir l'image de base pour les configurations d'exécution utilisant un CPU.
# 7. Importer Pipeline, PipelineData et PythonScriptStep pour construire et gérer des pipelines de données et des étapes de script Python dans Azure ML.
# 8. Importer os pour interagir avec le système de fichiers de l'OS hôte, souvent utilisé pour les chemins de fichiers et les variables d'environnement.

import os
from azureml.core import Workspace, Experiment, Dataset, Environment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import CondaDependencies, DEFAULT_CPU_IMAGE
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient


In [ ]:
# Configuration pour l'accès et la gestion des ressources Azure Blob Storage :
# 1. subscriptionId : Identifiant unique de l'abonnement Azure utilisé pour accéder aux services Azure.
# 2. resource_group : Nom du groupe de ressources dans Azure où sont stockées toutes les ressources liées.
# 3. account_name : Nom du compte de stockage Azure utilisé pour le stockage des blobs.
# 4. account_key : Clé d'accès au compte de stockage, utilisée pour authentifier les requêtes à Azure Blob Storage.
# 5. container_name : Nom du conteneur dans Azure Blob Storage où les blobs sont stockés.
# 6. local_file_path : Chemin local du fichier qui peut être téléchargé vers ou téléchargé depuis Azure Blob Storage.

subscriptionId = "*************"
resource_group = "*************"
account_name = "**************"
account_key = "********************"
container_name = "****************"
local_file_path = "/Users/mamoudousdiani/Documents/Certification/Python/donnees2"

creation d'un workspace

In [2]:
# Configuration de l'espace de travail Azure ML et des ressources de calcul :
# 1. Créer un espace de travail Azure ML avec création de groupe de ressources.
# 2. Sauvegarder la configuration de l'espace de travail localement.
# 3. Configurer un cluster de VM avec gestion dynamique des nœuds.
# 4. Vérifier et créer des cibles de calcul si elles n'existent pas, afficher les sorties du processus.

ws = Workspace.create(name = "pipeline_animaux_classification",
                      subscription_id = subscriptionId,
                      resource_group = resource_group,
                      location = "eastus",
                      create_resource_group = True)

ws.write_config(path=".azureml")

amlcomputes = {
    "cpu-pipeline-ac": {
        "vm_size": "STANDARD_DS3_V2",
        "min_nodes" : 0,
        "max_nodes" : 4,
        "idle_seconds_before_scaledown" : 240,
    }
}

for ct_name in amlcomputes :
    if ct_name not in ws.compute_targets:
        compute_config = AmlCompute.provisioning_configuration(**amlcomputes[ct_name])
        ct = ComputeTarget.create(ws, ct_name, compute_config)
        ct.wait_for_completion(show_output = True)

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Cleaning up past default Resource Group Deployments on the subscription to avoid limit of 10
Deleting past Resource Group Deployment with name: DeployResourceGroup-02604e130f
Deploying StorageAccount with name pipelinestorageb9852acea.
Deploying KeyVault with name pipelinekeyvault9bbc24ea.
Deploying Workspace with name pipeline_animaux_classification.
Deploying AppInsights with name pipelineinsightse9d81103.
Deployed AppInsights with name pipelineinsightse9d81103. Took 54.15 seconds.
Deployed Workspace with name pipeline_animaux_classification. Took 36.76 seconds.
InProgress......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


Creation d'une base de donnees

In [4]:
# Fonction pour téléverser un répertoire entier vers Azure Blob Storage :
# 1. Initialiser le client de service de blob avec les identifiants du compte.
# 2. Accéder au conteneur spécifié dans le compte de stockage.
# 3. Parcourir chaque fichier dans le répertoire local et ses sous-dossiers.
# 4. Téléverser chaque fichier au conteneur correspondant sous le même nom de blob.

def upload_directory_to_blob(account_name, account_key, container_name, local_directory_path):
    blob_service_client = BlobServiceClient(account_url=f"https://{account_name}.blob.core.windows.net", credential=account_key)

    container_client = blob_service_client.get_container_client(container_name)

    for foldername, subfolders, filenames in os.walk(local_directory_path):
        for filename in filenames:
            local_file_path = os.path.join(foldername, filename)
            blob_name = os.path.relpath(local_file_path, local_directory_path).replace("\\", "/")

            with open(local_file_path, "rb") as data:
                container_client.upload_blob(name=blob_name, data=data)

            print(f"Fichier {blob_name} uploadé avec succès dans le conteneur {container_name}.")


upload_directory_to_blob(account_name, account_key, container_name, local_file_path)


Fichier .DS_Store uploadé avec succès dans le conteneur pipeline-ac-db.
Fichier test/.DS_Store uploadé avec succès dans le conteneur pipeline-ac-db.
Fichier test/leopard/0298.png uploadé avec succès dans le conteneur pipeline-ac-db.
Fichier test/leopard/0267.png uploadé avec succès dans le conteneur pipeline-ac-db.
Fichier test/leopard/0501.png uploadé avec succès dans le conteneur pipeline-ac-db.
Fichier test/leopard/0515.png uploadé avec succès dans le conteneur pipeline-ac-db.
Fichier test/leopard/0273.png uploadé avec succès dans le conteneur pipeline-ac-db.
Fichier test/leopard/0529.png uploadé avec succès dans le conteneur pipeline-ac-db.
Fichier test/leopard/0918.png uploadé avec succès dans le conteneur pipeline-ac-db.
Fichier test/leopard/0924.png uploadé avec succès dans le conteneur pipeline-ac-db.
Fichier test/leopard/0930.png uploadé avec succès dans le conteneur pipeline-ac-db.
Fichier test/leopard/0703.png uploadé avec succès dans le conteneur pipeline-ac-db.
Fichier tes

Conception du pipeline
************************************************************

Configuration

In [181]:
# Accès à un espace de travail Azure ML et à une ressource de calcul :
# 1. Récupérer un espace de travail Azure ML existant par son nom, ID d'abonnement et groupe de ressources.
# 2. Définir la cible de calcul utilisée dans cet espace de travail pour des opérations ultérieures.

ws = Workspace.get(name='pipeline_animaux_classification', subscription_id=subscriptionId, resource_group='pipeline_ac')
compute_target= "cpu-pipeline-ac"

In [184]:
# Configuration d'une étape de pipeline pour la préparation des données dans Azure ML :
# 1. Définir un objet PipelineData pour stocker les données de sortie dans le magasin de données par défaut de l'espace de travail.
# 2. Configurer une étape de script Python qui exécute un script pour préparer les données, spécifiant le répertoire source, le nom du script, les arguments,
#    les données de sortie attendues, et la cible de calcul sur laquelle l'étape doit s'exécuter.

output_data = PipelineData("output_data", datastore=ws.get_default_datastore())
prepare_data_step = PythonScriptStep(
    source_directory='.',
    script_name="prepa_data.py",
    arguments=[ "--output-data", output_data],
    outputs=[output_data],
    compute_target=compute_target
)

In [185]:
# Configuration d'une étape de pipeline pour l'entraînement de modèle dans Azure ML :
# 1. Définir un dictionnaire pour les paramètres de script avec le chemin des données d'entrée.
# 2. Créer un objet PipelineData pour stocker le modèle résultant dans le magasin de données par défaut.
# 3. Configurer une étape de script Python pour l'entraînement du modèle, indiquant le répertoire source, les arguments (chemin des données et du modèle),
#    les entrées, les sorties, la cible de calcul et le nom du script.

script_params = {
    '--data-path': output_data,
    }
model_output = PipelineData("model_output", datastore=ws.get_default_datastore())
model_path="outputs/"
train_step = PythonScriptStep(
    source_directory='.',
    arguments=['--data-path', output_data, '--model-path', model_output],
    inputs=[output_data],
    outputs=[model_output],
    compute_target=compute_target,
    script_name='train.py'
)

In [186]:
# Configuration d'une étape de pipeline pour tester le modèle dans Azure ML :
# 1. Créer un objet PipelineData pour stocker les données de sortie du test dans le magasin de données par défaut.
# 2. Configurer une étape de script Python pour exécuter des tests sur le modèle, spécifiant le script, les arguments (répertoires du modèle et de sortie),
#    les entrées, les sorties, la cible de calcul, et le répertoire source.

test_output_data = PipelineData("test_output_data", datastore=ws.get_default_datastore())

test_step = PythonScriptStep(
    script_name="test_model.py",
    arguments=["--model-dir", model_output, "--output-data", test_output_data],
    inputs=[model_output],
    outputs=[test_output_data],
    compute_target=compute_target,
    source_directory="."
)

In [188]:
# Configuration et exécution d'un pipeline complet dans Azure ML :
# 1. Créer un pipeline avec les étapes préparées (préparation des données, entraînement, et test) et l'associer à l'espace de travail spécifié.
# 2. Créer une expérience sous le nom 'animal-classification-experiment' pour tracer l'exécution du pipeline.
# 3. Soumettre le pipeline à l'exécution dans le cadre de cette expérience et attendre la fin de l'exécution tout en affichant la sortie.

pipeline = Pipeline(workspace=ws, steps=[prepare_data_step, train_step, test_step])
experiment = Experiment(workspace=ws, name='animal-classification-experiment')
pipeline_run = experiment.submit(pipeline)
pipeline_run.wait_for_completion(show_output=True)

Created step prepa_data.py [4cc66126][40f4cf6f-bf10-482a-a962-25e0384708d7], (This step will run and generate new outputs)Created step train.py [11f860c6][de1c86d6-0148-40d3-b71c-b7df35af8587], (This step will run and generate new outputs)

Created step test_model.py [9c3de4d0][826e3262-4ee5-47ab-8175-388ed0d3f50a], (This step will run and generate new outputs)
Submitted PipelineRun d6efef9e-a5e6-410c-b3ea-cb961dcaa40c
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/d6efef9e-a5e6-410c-b3ea-cb961dcaa40c?wsid=/subscriptions/ae1240a3-81f1-4362-acce-875e12525303/resourcegroups/pipeline_ac/workspaces/pipeline_animaux_classification&tid=a02fa9c3-b2c8-4f00-b522-6edbf2b7c1f2
PipelineRunId: d6efef9e-a5e6-410c-b3ea-cb961dcaa40c
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/d6efef9e-a5e6-410c-b3ea-cb961dcaa40c?wsid=/subscriptions/ae1240a3-81f1-4362-acce-875e12525303/resourcegroups/pipeline_ac/workspaces/pipeline_animaux_classification&tid=a02fa9c3-b2c8-4f00-b52

'Finished'